## Interactive Modeling and Visualization of Physical and Energy Systems

### Problem 1

#### Goal: Create interactive dashboard that allows users to select day and month and output total energy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad, trapezoid
from ipywidgets import interactive, interact
import ipywidgets as widgets 
from scipy.integrate import odeint

In [2]:
# Read in full data and assign each column to a variable
fulldata = np.genfromtxt('LakeMead_2023.csv', skip_header = 3, delimiter = ',', dtype=int)

data = fulldata[:,1:]
month = data[:,0]
day = data[:,1]
hour = data[:,2]
min = data[:,3]
ghi = data[:,4]

In [3]:
# Create function that will allow users to input the day and month and output the total energy in kJ/sec
def energy(dy, mth):
    
    dmnth = np.where((day == dy) & (month == mth))[0] # Extract index where day and month match 
    ghiv = ghi[dmnth] # Returns GHI given index of day and month
    
    time = (hour[dmnth] * 3600 + min[dmnth] * 60) # Calculate time and convert to seconds
    
    total = trapezoid(ghiv, dx = 900)/1000 # Total energy using the trapezoid function (dx = 900 seconds - 15 minutes | Dividing by 1000 to convert)
    
    fig,ax = plt.subplots(figsize = (8,4))
    fig.suptitle('Solar Radiation')
    ax.plot(time,ghiv)
    ax.set_xlabel('Seconds')
    ax.set_ylabel('Global Horizontal Irradiance')
    
    print(f'Total Energy: {total} kJ/sec')

In [4]:
# Widgets to allow user to modify interactive plot
day_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=31,
    step=1,
    
)

month_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=12,
    step=1,
)

In [5]:
interactivePlot = interact(energy, dy = day_slider, mth = month_slider)

interactive(children=(IntSlider(value=1, description='dy', max=31, min=1), IntSlider(value=1, description='mth…

### Problem 2

#### Goal: Create an interactive dashboard to display a phase diagram and trajectory graph of a massless spring using odeint function

In [6]:
# Initial Values
k = 0.1
y1i = 0
y2i = 3

# First order equation
def ODE(y,t,k):
    y1, y2 = y
    y1p = y2
    y2p = -k*y1
    return [y1p, y2p]

# Function provides a way for users to modify k, initial velocity, and initial position
# Creating and plotting our phase diagram as well as our static plot
def plots(k, y1i, y2i):
    t = np.arange(0,10,0.1) 
    fig, ax = plt.subplots(figsize = (8,4))
    ax.set_title('Phase Diagram')
    ax.set_xlabel('Position')
    ax.set_ylabel('Velocity')
    ax.set_xlim(0,10)
    
    # Phase Diagram
    ph = odeint(ODE,[y1i,y2i],t,args=(k,)) # Solve the ODE with the odeint function
    
    y1v, y2v = np.arange(0,10.1,0.5), np.arange(0,10.1,0.5)
    y1s, y2s = np.meshgrid(y1v, y2v)
    
    # Finding our derivatives 
    y1p = y2s
    y2p = -k*y1s
    
    # Use quiver to plot points at (y1s,y2s) with arrows pointing in the direction of its derivative (y1p,y2p)
    plt.quiver(y1s, y2s, y1p, y2p)
    
    # Static plot of the springs position and velocity over time
    fig, ax = plt.subplots(figsize = (8,4))
    ax.set_title('Static Plot')
    ax.set_xlabel('Time')
    ax.set_ylabel('Position & Time')
    ax.set_xlim(0,9)
    
    plt.plot(t, ph[:,0], label = 'Position')
    plt.plot(t, ph[:,1], label = 'Velocity')
    
    plt.legend()

In [7]:
# Create widgets to modify k, initial position, and initial velocity
kwidg = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=10.0,
    step=0.1,
    description='K (constant):',
)

y1widg = widgets.FloatSlider(
    value=0,
    min=0,
    max=10.0,
    step=1,
    description='Position:',
)

y2widg = widgets.FloatSlider(
    value=3,
    min=-10.0,
    max=10.0,
    step=1,
    description='Velocity: ',
)

In [8]:
interactive = interact(plots, k = kwidg, y1i = y1widg, y2i = y2widg)

interactive(children=(FloatSlider(value=0.1, description='K (constant):', max=10.0, min=0.1), FloatSlider(valu…

#### Note: When widgets are modified, there might be more figures that are created. In other cases the graphs update just fine without duplications. Unsure how to fix when the outcome fluctuates.